Postprocessing with atooms
==========================

index

In this tutorial, we are going to show how to perform standard analysis
of molecular dynamics trajectories using the `atooms` package and its
`postprocessing` component. You can install `postprocessing` from pypi:
`pip install atooms-pp`.

Setup
-----

We start with a few imports. We will read a trajectory file in XYZ
format, so we also load the `TrajectoryXYZ` class from `atooms`.


In [ ]:
import matplotlib.pyplot as pl
import atooms.postprocessing as pp
from atooms.trajectory import TrajectoryXYZ 

We consider a sample trajectory, which can be downloaded from the
package repository. Let\'s store the path to this file in a variable for
later convenience


In [ ]:
from atooms.core.utils import download
download('https://framagit.org/atooms/postprocessing/raw/master/data/kalj-small.xyz', "/tmp")
path = '/tmp/kalj-small.xyz'

### Handling a trajectory

A trajectory is an object with many properties. To load a trajectory, we
create an instance of the class as follows


In [ ]:
th = TrajectoryXYZ(path)

The trajectory is a list-like object, in the sense that it can be
iterated upon and sliced. Each frame of the trajectory contains a
`System` object, which is a snaphot of the system at a given instant
(\"frame\") during the simulation. We have a look at the last frame


In [ ]:
print(th[-1])

To know how many frames we have


In [ ]:
print(len(th))

To clarify: a slice like `th[10: 12]` is a list of frames, not a
trajectory. It is possible to define a slice of a trajectory **as a
trajectory** instance by using the `Sliced` class decorator, see below.

If the trajectory contains metadata, these can be retrieved directly:


In [ ]:
from pprint import pprint
print("Timestep during the trajectory:", th.timestep)
print("Steps after which the 3rd frame was stored:", th.steps[2])
print("Additional metadata:")
pprint(th.metadata)

Analysis of the trajectory
--------------------------

Now that we have constructed a trajectory object, we can proceed to the
analysis. We are going to consider two main aspects of the analysis:

-   structural correlations
-   dynamical correlations

### Structural correlations

1.  Radial distribution function

    The radial distribution function $g(r)$ describes how the local
    density varies as a function of the distance from a reference
    particle. In liquids, one normally averages over all particles,
    obtaining a descriptor of the probability to find a second particle
    a distance $r$ from a given particle, relative to that of the ideal
    gas.

    For $N$ particles of the same type at density $\rho$ it is

    $$g(r)=\frac{1}{N\rho}\left\langle\sum_i^{N}\sum_{i\neq j}\delta(r-|\mathbf{r}_i-\mathbf{r}_j|)\right\rangle$$.

    Notice that the average number of particles with a distance $R$,
    i.e. the average **coordination number** $n(R)$, can be computed
    from the radial distribution function via integration in spherical
    coordinates (for 3D systems)

    $$n(R)=4\pi \rho\int_0^R g(r)r^2 dr$$

    In `postprocessing` the radial distribution function is a
    `Correlation` object that takes a trajectory as input. In order to
    compute the correlation function, we construct the object,
    specifying some parameters, and then run the calculation with the
    `compute()` method


In [ ]:
gr = pp.RadialDistributionFunction(th, dr=0.02)
gr.compute()

Once the calculation is performed, the radial distribution object `gr`
contains (like all correlators in `postprocessing`) two arrays:

-   the `grid` array contains the independent variable (or variables),
    binned according to our input parameters (in this case, the smallest
    space interval that we resolve, `dr`)
-   the `value` array contains the actual value of the computation, in
    this case the values of $g(r)$

We can directly plot the results with


In [ ]:
pl.plot(gr.grid, gr.value)
pl.xlabel("r")
pl.ylabel("g(r)")

![](gr.png)

You can show the results right away with the `show()` method, which
formats the labels automatically. The method returns a `matplotlib` axis
object for further customization.


In [ ]:
gr.show()

As we can see, the correlation function displays two narrow peaks around
$r=1$ and a broader peak further away. The presence of several peaks is
due to the fact that the system actually contains two types of
particles, noted $A$ and $B$.

We can compute separate distribution functions for the $A$ and $B$
particles and also the cross distribution funtion for the probability to
find a particle $B$ at distance $r$ from particle $A$ using the
`Partial` class.


In [ ]:
gr = pp.Partial(pp.RadialDistributionFunction, species=['A', 'B'], trajectory=th, dr=0.02)
gr.partial[('B', 'B')].dr = 0.05
gr.compute() 

Note how we modified the bin width `dr` for the $B$-$B$ correlations to
compensate for the reduced statistics for the minority species.

In this case, the result contains a dictionary `gr.partial`


In [ ]:
from pprint import pprint
pprint(gr.partial)

We plot all correlation functions


In [ ]:
for key,g in gr.partial.items(): 
    pl.plot(g.grid, g.value, label=str("".join(key)))
pl.legend()
pl.xlabel("r")
pl.ylabel(r"$g_{\alpha\beta}(r)$")

![](gr_ab.png)

Sometimes, it is useful to analyse only sections of a trajectory. To
this purpose, one can slice the trajectory using `atooms` and analyse
individual frames or subsets of frames.


In [ ]:
from atooms import trajectory
t =  trajectory.Sliced(th, slice(len(th)//2, len(th)))  # analyse only the 2nd half
gr = pp.RadialDistributionFunction(t, dr=0.02)
gr.compute()
pl.plot(gr.grid, gr.value)
pl.xlabel("r")
pl.ylabel("g(r)")

![](gr_slice.png)

### Dynamical correlations

1.  Mean square displacement

    We can also compute dynamical correlation functions. The simplest of
    such quantities is the mean squared displacement (MSD). This is
    defined as

    $$ \delta r^2(t)= \langle |\mathbf{r}(t-t_0) - \mathbf{r}(t_0)|^2\rangle$$

    The average is usually performed over all the $N$ particles and over
    multiple time origins $t_0$.

    The analysis process is now familiar. First we construct the msd
    object and then perform the calculation with `do()`.


In [ ]:
msd = pp.MeanSquareDisplacement(th)
msd.compute()

In [ ]:
pl.loglog(msd.grid, msd.value, '-o')
pl.xlabel("t")
pl.ylabel("MSD(t)");

![](msd.png)

Again, we can compute partial mean square displacements using the
`Partial` class


In [ ]:
msds = pp.Partial(pp.MeanSquareDisplacement, species=['A','B'], trajectory=th, norigins=100)
msds.compute()
pl.loglog(msds.partial['A'].grid, msds.partial['A'].value, '-o', label='A')
pl.loglog(msds.partial['B'].grid, msds.partial['B'].value, '-o', label='B')
pl.legend()
pl.xlabel("t")
pl.ylabel("MSD(t)")

![](msd_ab.png)

Self intermediate scattering function

We compute the self part of the intermediate scattering function (ISF)
at specific wave-vectors using a logarithmic time grid.


In [ ]:
from math import pi
import numpy
tgrid = numpy.logspace(0, 3, base=10)  # we must include t=0
isf = pp.Partial(pp.SelfIntermediateScattering, species=["A"], trajectory=th,
                 kgrid=[2*pi, 2.5*pi], nk=1, tgrid=tgrid)
isf.do()

To get some info on the parameters passed to compute the ISF, have a
look at the help for the base class with
`help(pp.fourierspace.FourierSpaceCorrelation)`

The ISF decays to zero at long times, as it should in an ergodic liquid.


In [ ]:
pl.semilogx(isf.partial['A'].grid[1], isf.partial['A'].value[0], '-o') 
pl.xlabel('t')
pl.ylabel('ISF')

![](isf.png)

### More correlation functions

Here is the full list of correlation functions currently available in
`postprocessing`, along with the corresponding classes:


In [ ]:
import inspect
import atooms.postprocessing as pp
for cls in inspect.getmembers(pp, inspect.isclass):    
    if issubclass(cls[1], pp.Correlation) \
       and cls[1] is not pp.Correlation \
       and not 'Fast' in cls[0] \
       and not 'Legacy' in cls[0] \
       and not 'Optimized' in cls[0] \
       and not 'Susceptibility' == cls[0]:
        print('- `{}`: {}'.format(cls[0], cls[1].long_name))

-   \`BondAngleDistribution\`: bond angle distribution
-   \`Chi4SelfOverlap\`: dynamic susceptibility of self overlap
-   \`CollectiveOverlap\`: collective overlap
-   \`IntermediateScattering\`: intermediate scattering function
-   \`MeanSquareDisplacement\`: mean square displacement
-   \`NonGaussianParameter\`: non-Gaussian parameter
-   \`RadialDistributionFunction\`: radial distribution function
-   \`S4ktOverlap\`: 4-point dynamic structure factor from self overlap
-   \`SelfIntermediateScattering\`: self intermediate scattering
    function
-   \`SelfOverlap\`: self overlap
-   \`SpectralDensity\`: spectral density
-   \`StructureFactor\`: structure factor
-   \`VelocityAutocorrelation\`: velocity autocorrelation

Some of them have multiple implementations (ex. `Fast` and `Legacy`),
which are picked at runtime depending on your platform. The fastest
implementation will be automatically picked up, if possible.

Writing your own correlation function
-------------------------------------

We can extend the correlation class to compute additional correlation
functions. The particle coordinates are loaded into the `_pos` instance
variable as a list of numpy arrays. Each numpy array is a
`(ndim, npart)` representation of the full particles\' coordinates in a
given frame of the trajectory. This is a gist of a new correlation
class.


In [ ]:
from collections import defaultdict
import numpy

def some_function(x, pos):
    return 0.0

class NewCorrelation(pp.correlation.Correlation):
    def __init__(self, trajectory, grid):
        pp.correlation.Correlation.__init__(self, trajectory, grid)
        self.phasespace = ["pos"]

    def _compute(self):
        raw = defaultdict(list)
        for i in range(len(self._pos)):
            for x in self.grid: 
                raw[x].append(some_function(x, self._pos[i]))        
        self.value = [numpy.mean(raw[x]) for x in raw]

In [ ]:
nw = NewCorrelation(th, [0.0, 1.0])
nw.compute()
print(nw.value)

Storing results and analysis
----------------------------

The correlation functions can be written to files, using the `write()`
method.


In [ ]:
msd = pp.MeanSquareDisplacement(th)
msd.compute()
msd.write()

The output file path is interpolated using the variable
`core.pp_output_path`


In [ ]:
print({} gives {}'.format(pp.core.pp_output_path, msd._output_file))

To change the output file pattern just modify the string. You can use
any `Correlation` attribute enclosed in brackets to parameterize the
output.

The output file is a simple columnar text file containing the grid and
values of the correlation function.

Some correlation functions may implement some basic analysis as well


In [ ]:
msd.analyze()
pprint(msd.analysis)

If `analyze()` is called before `compute()`, the above dictionary will
be written in the output file as well.

Finally, you can perform all the steps above (`compute`, `analyze`,
`write`) in one sweep like this


In [ ]:
msd.do()